In [1]:
import os
import platform

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# import matplotlib.pyplot as plt
import tqdm
import cv2
import pandas as pd

from street_signs_identify.utils.x_image_loader import XImageLoader
import street_signs_identify.utils.check_overlap as co

from street_signs_identify.detector.craft_detector import CRAFTDetector

In [2]:
folder_name = "train"
data_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{folder_name}'
output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc\\{folder_name}'
output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_gt\\{folder_name}'
output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_filter\\{folder_name}'
output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_craft\\{folder_name}'

if platform.system() == "Linux":
    folder_name = "./dataset/"
    data_path = os.path.join(folder_name, "2Penghu")
    output_folder = os.path.join(folder_name, "cropped_2Penghu")
    output_folder_gt = os.path.join(output_folder, "ref_gt")
    output_folder_filtered_pred = os.path.join(output_folder, "filtered_pred")
    output_folder_craft_pred = os.path.join(output_folder, "craft_pred")


os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_gt, exist_ok=True)
os.makedirs(output_folder_filtered_pred, exist_ok=True)
os.makedirs(output_folder_craft_pred, exist_ok=True)

In [3]:
detector = CRAFTDetector()
x_image_loader = XImageLoader(data_path, annotation_file="labels.csv")

Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232747579_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232747926_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232748282_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232748649_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232749037_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232749458_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232749923_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232750457_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232751091_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232751944_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232753091_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Pengh

In [4]:
x_image_loader._info

,raw_filename,filename,category,box,label,polygon,x0,y0,x1,y1
0,220720_225837855_Camera_0.jpg,NaN,3,"[(1853, 861), (1861, 927), (1996, 951), (1987,...",永順\nYongshu,"POLYGON ((1853 861, 1861 927, 1996 951, 1987 8...",1853,861,1996,951
1,220720_225953347_Camera_0.jpg,NaN,3,"[(1469, 789), (1473, 842), (1616, 848), (1608,...",永順街\nYingshun St.,"POLYGON ((1469 789, 1473 842, 1616 848, 1608 7...",1469,789,1616,848
2,220720_225953866_Camera_0.jpg,NaN,3,"[(1565, 701), (1574, 766), (1738, 779), (1727,...",永順街\nYongshun St.,"POLYGON ((1565 701, 1574 766, 1738 779, 1727 7...",1565,701,1738,779
3,220720_225954302_Camera_0.jpg,NaN,3,"[(1706, 572), (1722, 647), (1907, 675), (1889,...",永順街\nYongshun St.,"POLYGON ((1706 572, 1722 647, 1907 675, 1889 6...",1706,572,1907,675
4,220720_230048513_Camera_0.jpg,NaN,3,"[(1615, 739), (1618, 799), (1762, 812), (1760,...",同義和路\nTonghe Rd.,"POLYGON ((1615 739, 1618 799, 1762 812, 1760 7...",1615,739,1762,812
...,...,...,...,...,...,...,...,...,...,...
256,220720_234008018_Camera_0.jpg,NaN,3,"[(1757, 284), (1408, 246), (1381, 160), (1725,...",中正路\nZhongzheng Rd.,"POLYGON ((1757 284, 1408 246, 1381 160, 1725 2...",1381,160,1757,284
257,220720_234136955_Camera_0.jpg,NaN,3,"[(1639, 761), (1475, 749), (1465, 687), (1627,...",樹德路\nShude Rd.,"POLYGON ((1639 761, 1475 749, 1465 687, 1627 7...",1465,687,1639,761
258,220720_234137338_Camera_0.jpg,NaN,3,"[(1770, 658), (1579, 629), (1561, 552), (1751,...",樹德路\nShude Rd.,"POLYGON ((1770 658, 1579 629, 1561 552, 1751 5...",1561,552,1770,658
259,220720_234137732_Camera_0.jpg,NaN,3,"[(1945, 503), (1726, 431), (1697, 343), (1916,...",樹德路\nShude Rd.,"POLYGON ((1945 503, 1726 431, 1697 343, 1916 4...",1697,343,1945,503


In [8]:
global_index = 0

label_df = pd.DataFrame(columns=["filename", "category", "words", "src_file", "gt_index", "detected_index", "IOU"])

for image_name in tqdm.tqdm(x_image_loader._image_names):
    gt_image = x_image_loader[image_name]
    detected_image = detector(gt_image.image)
    if detected_image is None:
        continue
    
    overlapping_info = co.check_overlapping_area(gt_image.info, detected_image.info)
    if overlapping_info is None:
        continue

    # save gt plot
    gt_img = gt_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_gt, image_name), cv2.cvtColor(gt_img, cv2.COLOR_RGB2BGR))
    
    # save craft plot
    det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_craft_pred, image_name), cv2.cvtColor(det_img, cv2.COLOR_RGB2BGR))

    filtered_image = detected_image.keeps_index(overlapping_info["index_2nd"])

    filtered_det_img = filtered_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_filtered_pred, image_name), cv2.cvtColor(filtered_det_img, cv2.COLOR_RGB2BGR))

    for row_idx, row in overlapping_info.iterrows():
        gt_index, detected_index = row["index_1st"], row["index_2nd"]
        detected_instance = detected_image[detected_index]
        if not detected_instance.legal:
            continue
        
        label_df.loc[global_index, "filename"] = f"{global_index}.jpg"
        label_df.loc[global_index, "src_file"] = image_name
        label_df.loc[global_index, "category"] = gt_image.info.loc[gt_index, "category"]
        label_df.loc[global_index, "words"] = gt_image.info.loc[gt_index, "label"]
        label_df.loc[global_index, "IOU"] = row["IOU"]
        label_df.loc[global_index, "gt_index"] = int(gt_index)
        label_df.loc[global_index, "detected_index"] = int(detected_index)

        crapped_img = detected_instance.image
        cv2.imwrite(os.path.join(output_folder, f"{global_index}.jpg"), cv2.cvtColor(crapped_img, cv2.COLOR_RGB2BGR))
        global_index+=1




label_df.to_csv(os.path.join(output_folder, "labels.csv"), encoding = 'big5')

100%|██████████| 162/162 [03:00<00:00,  1.11s/it]


In [9]:
for src_file, dg in label_df.groupby("src_file"):
    for detected_idx, sub_dg in dg.groupby("detected_index"):
        sorted_index = sub_dg["IOU"].sort_index()
        label_df.loc[sorted_index, "priority"] = range(len(sorted_index))

  filename category        words                       src_file gt_index  \
0    0.jpg        3  永順\nYongshu  220720_225837855_Camera_0.jpg        0   
1    1.jpg        3  永順\nYongshu  220720_225837855_Camera_0.jpg        0   

  detected_index       IOU  
0              0  0.215776  
1              5  0.093024  
  filename category              words                       src_file  \
2    2.jpg        3  永順街\nYingshun St.  220720_225953347_Camera_0.jpg   
3    3.jpg        3  永順街\nYingshun St.  220720_225953347_Camera_0.jpg   

  gt_index detected_index       IOU  
2        0              0  0.510323  
3        0              1  0.167003  
  filename category              words                       src_file  \
4    4.jpg        3  永順街\nYongshun St.  220720_225953866_Camera_0.jpg   
5    5.jpg        3  永順街\nYongshun St.  220720_225953866_Camera_0.jpg   
6    6.jpg        3  永順街\nYongshun St.  220720_225953866_Camera_0.jpg   

  gt_index detected_index       IOU  
4        0         